In [1]:
# Tratamiento base de datos
import pandas as pd
import copy

# Acceder a directorio
from os import listdir
from os.path import isfile, join

# Esconder warnings
import warnings
warnings.filterwarnings('ignore')

# Estadísticas
from scipy.stats import skew

# Lectura de bases de datos

In [2]:
# ======================== #
# Order Book Files Bitcoin #
# ======================== #

obBtc = 'C:/Users/victo/Desktop/UNIVERSIDAD/MASTER/TFM/Order_Book/BTCBUSD'
obBtcFiles = [f for f in listdir(obBtc) if isfile(join(obBtc, f))]

# ========================= #
# Order Book Files Ethereum #
# ========================= #

obEth = 'C:/Users/victo/Desktop/UNIVERSIDAD/MASTER/TFM/Order_Book/ETHBUSD'
obEthFiles = [f for f in listdir(obEth) if isfile(join(obEth, f))]

# ==================== #
# Prices Files Bitcoin #
# ==================== #

# prBtc = 'C:/Users/victo/Desktop/UNIVERSIDAD/MASTER/TFM/Prices/BTCBUSD'
# prBtcFiles = [f for f in listdir(prBtc) if isfile(join(prBtc, f))]

# ===================== #
# Prices Files Ethereum #
# ===================== #

# prEth = 'C:/Users/victo/Desktop/UNIVERSIDAD/MASTER/TFM/Prices/ETHBUSD'
# prEthFiles = [f for f in listdir(prEth) if isfile(join(prEth, f))]

# Tratamiento de bases de datos

In [3]:
def preProcesado(df):
    # ============= # 
    # Decodificamos #
    # ============= #
    
    # b'bids', b'2021-04-30, 13:24:51', b'BTCBUSD'
    clms = ['side', 'date', 'crpt']
    
    for cl in clms:
        df[cl] = df[cl].str.decode('utf-8')
    
    # ==== # 
    # PrQt #
    # ==== #
    
    # Price * quantity
    df['PrQt'] = round(df['price'] * df['quantity'], 3)
    
    # ================ # 
    # Split & midprice #
    # ================ #
    
    dfBids = df[df['side'] == 'bids']
    dfAsks = df[df['side'] == 'asks']
    
    midprice = (np.max(dfBids.price) + np.min(dfAsks.price)) / 2
    
    return(dfBids, dfAsks, round(midprice, 2))

In [4]:
def addDate(df):
    # ======== # 
    # Add date #
    # ======== #
    Dn = []
    
    # date
    date = pd.to_datetime(df['date'], infer_datetime_format = True)[0]
    Dn.append(date)
    
    # minutes since 00:00 in:
    # Beijing
    minBjg = (date +  pd.to_timedelta(6, unit='h')).hour * 60 + date.minute
    Dn.append(minBjg)
    
    return(Dn)

In [5]:
def CreateBids(dfBids, cubes, midprice):
    # ========================= #
    # Decrement midprice: bids #
    # ========================= #
    DecrArray = [round(midprice * (1 - d / 100), 2) for d in cubes]
         
    # ¿Cuántos USD [precio * cantidad] se están pujando a un porcentaje de precio DecrArray[i] inferior al midprice?   
    BArray = []
                
    for i in DecrArray:
        BidPrQt = np.sum(dfBids[dfBids['price'] >= i].PrQt)
        BArray.append(round(BidPrQt, 3))
                    
    return(BArray)

In [6]:
def CreateAsks(dfAsks, cubes, midprice):
    # ========================= #
    # Increment midprice: asks #
    # ========================= #
    IncrArray = [round(midprice * ((100 + d) / 100), 2) for d in cubes]
                
    # ¿Cuántos USD [precio * cantidad] se están ofertando a un porcentaje de precio DecrArray[i] superior al midprice?
    AArray = []
                
    for i in IncrArray:
        AskPrQt = np.sum(dfAsks[dfAsks['price'] <= i].PrQt)
        AArray.append(round(AskPrQt, 3))
                
    return(AArray)

In [7]:
def WeightedFeatures(dfSide, side):
    wf = []
    
    # Proporción (n / 12502500)
    if side == 'bid':
        num = list(range(1, 5001, 1))
    else:
        num = list(range(5000, 0, -1))
    
    # Weights
    w = num / np.sum(list(range(1, 5001)))
    
    # ============= # 
    # Weighted Mean #
    # ============= #
    WghtMean = round(np.average(dfSide['PrQt'], weights = w))
    wf.append(WghtMean)
    
    # =========================== # 
    # Weighted Standard Deviation #
    # =========================== #
    WghtStd = round(np.sqrt(np.average((dfSide['PrQt'] - WghtMean) ** 2, weights = w)))
    wf.append(WghtStd)
    
    return(wf)

In [8]:
def DistrFeatures(dfSide, side):
    # ============ # 
    # Distribution #
    # ============ #
    
    Distr = []
    
    if side == 'bid':
        # ¿Qué volumen se quiere comprar a un precio superior al precio medio?
        BidMean = np.sum(dfSide[dfSide['price'] >= np.mean(dfSide['price'])].PrQt)
        Distr.append(BidMean)
        
        # ¿Qué volumen se quiere comprar a un precio superior al precio mediano?
        BidMedian = np.sum(dfSide[dfSide['price'] >= np.median(dfSide['price'])].PrQt)
        Distr.append(BidMedian)
        
        # ¿Qué volumen se quiere comprar a un precio superior al precio Q1?
        BidQ1 = np.sum(dfSide[dfSide['price'] >= np.percentile(dfSide['price'], 25)].PrQt)
        Distr.append(BidQ1)
        
        # ¿Qué volumen se quiere comprar a un precio superior al precio Q3?
        BidQ3 = np.sum(dfSide[dfSide['price'] >= np.percentile(dfSide['price'], 75)].PrQt)
        Distr.append(BidQ3)
        
        # Skewness
        BidSkew = skew(dfSide['PrQt'])
        Distr.append(BidSkew)
    else:
        # ¿Qué volumen se quiere comprar a un precio inferior al precio medio?
        AskMean = np.sum(dfSide[dfSide['price'] <= np.mean(dfSide['price'])].PrQt)
        Distr.append(AskMean)
        
        # ¿Qué volumen se quiere comprar a un precio inferior al precio mediano?
        AskMedian = np.sum(dfSide[dfSide['price'] <= np.median(dfSide['price'])].PrQt)
        Distr.append(AskMedian)
        
        # ¿Qué volumen se quiere comprar a un precio inferior al precio Q1?
        AskQ1 = np.sum(dfSide[dfSide['price'] <= np.percentile(dfSide['price'], 25)].PrQt)
        Distr.append(AskQ1)
        
        # ¿Qué volumen se quiere comprar a un precio inferior al precio Q3?
        AskQ3 = np.sum(dfSide[dfSide['price'] <= np.percentile(dfSide['price'], 75)].PrQt)
        Distr.append(AskQ3)
        
        # Skewness
        AskSkew = skew(dfSide['PrQt'])
        Distr.append(AskSkew)        
    
    return(Distr)

In [9]:
def CreateSlope(offers, cubes, midprice):
    
    SlopeArray = offers / (cubes * midprice)
        
    return(list(SlopeArray))

In [10]:
def addOrderBook(dfBids, dfAsks, cubes, midprice, crp):    
    # ========================= # 
    # Bids variables (grupo II) #
    # ========================= #
    bids = CreateBids(dfBids, cubes, midprice)
    
    if crp == 'btc':
        # =============================================== # 
        # Weighted Mean & Weighted Std: bids (grupo IV.1) #
        # =============================================== #
        WgtFeatBids = WeightedFeatures(dfBids, side = 'bid')
    
    # ================================= # 
    # Slope variables: bids (grupo V.1) #
    # ================================= #
    SlopeBids = CreateSlope(bids[0:30:2], np.array(cubes[0:30:2]) / 100, midprice)
    
    if crp == 'btc':
        # =============================== # 
        # Distribution: bids (grupo VI.1) #
        # =============================== #
        DistrBids = DistrFeatures(dfBids, side = 'bid')
    
    # ========================== # 
    # Asks variables (grupo III) #
    # ========================== #
    asks = CreateAsks(dfAsks, cubes, midprice)
    
    if crp == 'btc':
        # =============================================== # 
        # Weighted Mean & Weighted Std: asks (grupo IV.2) #
        # =============================================== #
        WgtFeatAsks = WeightedFeatures(dfAsks, side = 'ask')
    
    # ================================= # 
    # Slope variables: asks (grupo V.2) #
    # ================================= #
    SlopeAsks = CreateSlope(asks[0:30:2], np.array(cubes[0:30:2]) / 100, midprice)
    
    if crp == 'btc':
        # =============================== # 
        # Distribution: asks (grupo VI.2) #
        # =============================== #
        DistrAsks = DistrFeatures(dfAsks, side = 'ask')
    
    if crp == 'btc':
        # Bitcoin
        ob = bids + WgtFeatBids + SlopeBids + DistrBids + asks + WgtFeatAsks + SlopeAsks + DistrAsks
        
    else:
        # Ethereum
        ob = bids + SlopeBids + asks  + SlopeAsks
    
    return(ob)

In [11]:
def Tipo1(dateArray, midpriceArray, y):
    
    # Fecha. Eliminamos los dos primeros registros por "memoria"
    D = pd.DataFrame(dateArray).iloc[2:, 0]
    
    # Creamos los vectores con memoria de orden 1 y orden 2
    subList = np.array([midpriceArray[n:n+3][::-1] for n in range(0, len(midpriceArray) - 2)])
    
    # Pandas predictors
    H = pd.DataFrame(subList)
    
    # Concatenamos
    X = pd.concat([D.reset_index(drop = True), H], axis=1)
    X.columns = ['date', 'MPT0', 'MPT1', 'MPT2']
    
    # Tendencia
    # La fila 0 en el dataset corresponde al midprice con índice 2
    # La variable predictora será la del índice 6 que mide el cambio entre los midprice 6-7
    # 0 (2) --> 6 (6-7)
    
    # Eliminamos desde N-5 hasta N porque no existe la variable a predecir
    df = X.drop(X.index[len(X)-5:len(X)])
    
    # Tendencia. Eliminamos los dos primeros registros por "memoria"
    df['tendencia'] = y[2:]
    
    return(df)

In [12]:
def Tipo2(Dn, Bn, y):
    D = pd.DataFrame(Dn)
    D.columns = ['date', 'minBjg'] 
    
    B = pd.DataFrame(Bn)
    B.columns = ['bid0.1', 'bid0.2', 'bid0.3', 'bid0.4', 'bid0.5',
                 'bid0.6', 'bid0.7', 'bid0.8', 'bid0.9', 'bid1.0',
                 'bid1.1', 'bid1.2', 'bid1.3', 'bid1.4', 'bid1.5', 
                 'bid1.6', 'bid1.8', 'bid2.0', 'bid2.2', 'bid2.4', 
                 'bid2.6', 'bid2.8', 'bid3.0', 'bid3.2', 'bid3.6', 
                 'bid4.0', 'bid4.4', 'bid4.8', 'bid5.4', 'bid6.0', 
                 'WMBid', 'WStBid',
                 'BidSlp0.1', 'BidSlp0.3', 'BidSlp0.5', 'BidSlp0.7', 
                 'BidSlp0.9', 'BidSlp1.1', 'BidSlp1.3', 'BidSlp1.5', 
                 'BidSlp1.8', 'BidSlp2.2', 'BidSlp2.6', 'BidSlp3.0', 
                 'BidSlp3.6', 'BidSlp4.4', 'BidSlp5.4',
                 'BidMean', 'BidMedian', 'BidQ1', 'BidQ3', 'BidSkew',
                 'ask0.1', 'ask0.2', 'ask0.3', 'ask0.4', 'ask0.5', 
                 'ask0.6', 'ask0.7', 'ask0.8', 'ask0.9', 'ask1.0',
                 'ask1.1', 'ask1.2', 'ask1.3', 'ask1.4', 'ask1.5', 
                 'ask1.6', 'ask1.8', 'ask2.0', 'ask2.2', 'ask2.4', 
                 'ask2.6', 'ask2.8', 'ask3.0', 'ask3.2', 'ask3.6', 
                 'ask4.0', 'ask4.4', 'ask4.8', 'ask5.4', 'ask6.0',
                 'WMAsk', 'WStAsk',
                 'AskSlp0.1', 'AskSlp0.3', 'AskSlp0.5', 'AskSlp0.7', 
                 'AskSlp0.9', 'AskSlp1.1', 'AskSlp1.3', 'AskSlp1.5',
                 'AskSlp1.8', 'AskSlp2.2', 'AskSlp2.6', 'AskSlp3.0', 
                 'AskSlp3.6', 'AskSlp4.4', 'AskSlp5.4',                 
                 'AskMean', 'AskMedian', 'AskQ1', 'AskQ3', 'AskSkew']
     
    X = pd.concat([D.reset_index(drop = True), B], axis=1)
    
    # Eliminamos desde N-5 hasta N porque no existe la variable a predecir
    df = X.drop(X.index[len(X)-5:len(X)])
        
    # Variable respuesta
    df['tendencia'] = y
        
    return(df)

In [23]:
def createMatrix(tipo, BTCOB, cubes):
    
    # tipo: planteamiento 1, 2, 3 o 4
    # BTCOB: Bitcoin Order Book Files
    # cubes: nº de intervalos y tamaño para definir las variables: bids, asks y slope 
    
    midpriceArray = [] # for Bitcoin
    dateArray = []
    OBArray = [] # for Bitcoin
    
    for i in range(len(BTCOB)):
        print('File:', i, '/32562')
        df_ObBtc = pd.read_feather('Order_Book/BTCBUSD/' + BTCOB[i])

        # ============ # 
        # Preprocesado #
        # ============ #
        dfBids, dfAsks, midprice = preProcesado(df_ObBtc)
        midpriceArray.append(midprice)
            
        # =================== # 
        # Add date: Create Dn #
        # =================== #
        date = addDate(df_ObBtc)
        dateArray.append(date)

        if tipo != 1:
            # ============================== # 
            # Order Book features: Create Bn #
            # ============================== #
            ob = addOrderBook(dfBids, dfAsks, cubes, midprice, crp = 'btc')
            OBArray.append(ob)
                  
    # ======== #
    # tendency #
    # ======== #
    
    # Diferencias en tendencia (a 5 minutos)
    ydiffs = [midpriceArray[i + 5] - midpriceArray[i] for i in range(0, len(midpriceArray) - 5)]

    # Creamos tendencia {-1: baja, 1: sube}
    y = []
    for l in range(0, len(ydiffs)):
        if ydiffs[l] > 0:
            y.append(1)
            
        else:
            y.append(-1)
    
    if tipo == 1:
        # =============== #
        # Planteamiento 1 #
        # =============== #
        df = Tipo1(dateArray, midpriceArray, y)
        return(df)
    
    else:
        # =============== #
        # Planteamiento 2 #
        # =============== #
        df = Tipo2(dateArray, OBArray, y)
        return(df)

# Planteamiento 1

In [16]:
# Creamos los datos del planteamiento 1
df1 = createMatrix(tipo = 1, BTCOB = obBtcFiles, cubes = None)

File: 0 /32562
File: 1 /32562
File: 2 /32562
File: 3 /32562
File: 4 /32562
File: 5 /32562
File: 6 /32562
File: 7 /32562
File: 8 /32562
File: 9 /32562
File: 10 /32562
File: 11 /32562
File: 12 /32562
File: 13 /32562
File: 14 /32562
File: 15 /32562
File: 16 /32562
File: 17 /32562
File: 18 /32562
File: 19 /32562
File: 20 /32562
File: 21 /32562
File: 22 /32562
File: 23 /32562
File: 24 /32562
File: 25 /32562
File: 26 /32562
File: 27 /32562
File: 28 /32562
File: 29 /32562
File: 30 /32562
File: 31 /32562
File: 32 /32562
File: 33 /32562
File: 34 /32562
File: 35 /32562
File: 36 /32562
File: 37 /32562
File: 38 /32562
File: 39 /32562
File: 40 /32562
File: 41 /32562
File: 42 /32562
File: 43 /32562
File: 44 /32562
File: 45 /32562
File: 46 /32562
File: 47 /32562
File: 48 /32562
File: 49 /32562
File: 50 /32562
File: 51 /32562
File: 52 /32562
File: 53 /32562
File: 54 /32562
File: 55 /32562
File: 56 /32562
File: 57 /32562
File: 58 /32562
File: 59 /32562
File: 60 /32562
File: 61 /32562
File: 62 /32562
Fi

File: 490 /32562
File: 491 /32562
File: 492 /32562
File: 493 /32562
File: 494 /32562
File: 495 /32562
File: 496 /32562
File: 497 /32562
File: 498 /32562
File: 499 /32562
File: 500 /32562
File: 501 /32562
File: 502 /32562
File: 503 /32562
File: 504 /32562
File: 505 /32562
File: 506 /32562
File: 507 /32562
File: 508 /32562
File: 509 /32562
File: 510 /32562
File: 511 /32562
File: 512 /32562
File: 513 /32562
File: 514 /32562
File: 515 /32562
File: 516 /32562
File: 517 /32562
File: 518 /32562
File: 519 /32562
File: 520 /32562
File: 521 /32562
File: 522 /32562
File: 523 /32562
File: 524 /32562
File: 525 /32562
File: 526 /32562
File: 527 /32562
File: 528 /32562
File: 529 /32562
File: 530 /32562
File: 531 /32562
File: 532 /32562
File: 533 /32562
File: 534 /32562
File: 535 /32562
File: 536 /32562
File: 537 /32562
File: 538 /32562
File: 539 /32562
File: 540 /32562
File: 541 /32562
File: 542 /32562
File: 543 /32562
File: 544 /32562
File: 545 /32562
File: 546 /32562
File: 547 /32562
File: 548 /325

File: 977 /32562
File: 978 /32562
File: 979 /32562
File: 980 /32562
File: 981 /32562
File: 982 /32562
File: 983 /32562
File: 984 /32562
File: 985 /32562
File: 986 /32562
File: 987 /32562
File: 988 /32562
File: 989 /32562
File: 990 /32562
File: 991 /32562
File: 992 /32562
File: 993 /32562
File: 994 /32562
File: 995 /32562
File: 996 /32562
File: 997 /32562
File: 998 /32562
File: 999 /32562
File: 1000 /32562
File: 1001 /32562
File: 1002 /32562
File: 1003 /32562
File: 1004 /32562
File: 1005 /32562
File: 1006 /32562
File: 1007 /32562
File: 1008 /32562
File: 1009 /32562
File: 1010 /32562
File: 1011 /32562
File: 1012 /32562
File: 1013 /32562
File: 1014 /32562
File: 1015 /32562
File: 1016 /32562
File: 1017 /32562
File: 1018 /32562
File: 1019 /32562
File: 1020 /32562
File: 1021 /32562
File: 1022 /32562
File: 1023 /32562
File: 1024 /32562
File: 1025 /32562
File: 1026 /32562
File: 1027 /32562
File: 1028 /32562
File: 1029 /32562
File: 1030 /32562
File: 1031 /32562
File: 1032 /32562
File: 1033 /325

File: 1442 /32562
File: 1443 /32562
File: 1444 /32562
File: 1445 /32562
File: 1446 /32562
File: 1447 /32562
File: 1448 /32562
File: 1449 /32562
File: 1450 /32562
File: 1451 /32562
File: 1452 /32562
File: 1453 /32562
File: 1454 /32562
File: 1455 /32562
File: 1456 /32562
File: 1457 /32562
File: 1458 /32562
File: 1459 /32562
File: 1460 /32562
File: 1461 /32562
File: 1462 /32562
File: 1463 /32562
File: 1464 /32562
File: 1465 /32562
File: 1466 /32562
File: 1467 /32562
File: 1468 /32562
File: 1469 /32562
File: 1470 /32562
File: 1471 /32562
File: 1472 /32562
File: 1473 /32562
File: 1474 /32562
File: 1475 /32562
File: 1476 /32562
File: 1477 /32562
File: 1478 /32562
File: 1479 /32562
File: 1480 /32562
File: 1481 /32562
File: 1482 /32562
File: 1483 /32562
File: 1484 /32562
File: 1485 /32562
File: 1486 /32562
File: 1487 /32562
File: 1488 /32562
File: 1489 /32562
File: 1490 /32562
File: 1491 /32562
File: 1492 /32562
File: 1493 /32562
File: 1494 /32562
File: 1495 /32562
File: 1496 /32562
File: 1497

File: 1905 /32562
File: 1906 /32562
File: 1907 /32562
File: 1908 /32562
File: 1909 /32562
File: 1910 /32562
File: 1911 /32562
File: 1912 /32562
File: 1913 /32562
File: 1914 /32562
File: 1915 /32562
File: 1916 /32562
File: 1917 /32562
File: 1918 /32562
File: 1919 /32562
File: 1920 /32562
File: 1921 /32562
File: 1922 /32562
File: 1923 /32562
File: 1924 /32562
File: 1925 /32562
File: 1926 /32562
File: 1927 /32562
File: 1928 /32562
File: 1929 /32562
File: 1930 /32562
File: 1931 /32562
File: 1932 /32562
File: 1933 /32562
File: 1934 /32562
File: 1935 /32562
File: 1936 /32562
File: 1937 /32562
File: 1938 /32562
File: 1939 /32562
File: 1940 /32562
File: 1941 /32562
File: 1942 /32562
File: 1943 /32562
File: 1944 /32562
File: 1945 /32562
File: 1946 /32562
File: 1947 /32562
File: 1948 /32562
File: 1949 /32562
File: 1950 /32562
File: 1951 /32562
File: 1952 /32562
File: 1953 /32562
File: 1954 /32562
File: 1955 /32562
File: 1956 /32562
File: 1957 /32562
File: 1958 /32562
File: 1959 /32562
File: 1960

...

File: 30640 /32562
File: 30641 /32562
File: 30642 /32562
File: 30643 /32562
File: 30644 /32562
File: 30645 /32562
File: 30646 /32562
File: 30647 /32562
File: 30648 /32562
File: 30649 /32562
File: 30650 /32562
File: 30651 /32562
File: 30652 /32562
File: 30653 /32562
File: 30654 /32562
File: 30655 /32562
File: 30656 /32562
File: 30657 /32562
File: 30658 /32562
File: 30659 /32562
File: 30660 /32562
File: 30661 /32562
File: 30662 /32562
File: 30663 /32562
File: 30664 /32562
File: 30665 /32562
File: 30666 /32562
File: 30667 /32562
File: 30668 /32562
File: 30669 /32562
File: 30670 /32562
File: 30671 /32562
File: 30672 /32562
File: 30673 /32562
File: 30674 /32562
File: 30675 /32562
File: 30676 /32562
File: 30677 /32562
File: 30678 /32562
File: 30679 /32562
File: 30680 /32562
File: 30681 /32562
File: 30682 /32562
File: 30683 /32562
File: 30684 /32562
File: 30685 /32562
File: 30686 /32562
File: 30687 /32562
File: 30688 /32562
File: 30689 /32562
File: 30690 /32562
File: 30691 /32562
File: 30692 

File: 31072 /32562
File: 31073 /32562
File: 31074 /32562
File: 31075 /32562
File: 31076 /32562
File: 31077 /32562
File: 31078 /32562
File: 31079 /32562
File: 31080 /32562
File: 31081 /32562
File: 31082 /32562
File: 31083 /32562
File: 31084 /32562
File: 31085 /32562
File: 31086 /32562
File: 31087 /32562
File: 31088 /32562
File: 31089 /32562
File: 31090 /32562
File: 31091 /32562
File: 31092 /32562
File: 31093 /32562
File: 31094 /32562
File: 31095 /32562
File: 31096 /32562
File: 31097 /32562
File: 31098 /32562
File: 31099 /32562
File: 31100 /32562
File: 31101 /32562
File: 31102 /32562
File: 31103 /32562
File: 31104 /32562
File: 31105 /32562
File: 31106 /32562
File: 31107 /32562
File: 31108 /32562
File: 31109 /32562
File: 31110 /32562
File: 31111 /32562
File: 31112 /32562
File: 31113 /32562
File: 31114 /32562
File: 31115 /32562
File: 31116 /32562
File: 31117 /32562
File: 31118 /32562
File: 31119 /32562
File: 31120 /32562
File: 31121 /32562
File: 31122 /32562
File: 31123 /32562
File: 31124 

File: 31504 /32562
File: 31505 /32562
File: 31506 /32562
File: 31507 /32562
File: 31508 /32562
File: 31509 /32562
File: 31510 /32562
File: 31511 /32562
File: 31512 /32562
File: 31513 /32562
File: 31514 /32562
File: 31515 /32562
File: 31516 /32562
File: 31517 /32562
File: 31518 /32562
File: 31519 /32562
File: 31520 /32562
File: 31521 /32562
File: 31522 /32562
File: 31523 /32562
File: 31524 /32562
File: 31525 /32562
File: 31526 /32562
File: 31527 /32562
File: 31528 /32562
File: 31529 /32562
File: 31530 /32562
File: 31531 /32562
File: 31532 /32562
File: 31533 /32562
File: 31534 /32562
File: 31535 /32562
File: 31536 /32562
File: 31537 /32562
File: 31538 /32562
File: 31539 /32562
File: 31540 /32562
File: 31541 /32562
File: 31542 /32562
File: 31543 /32562
File: 31544 /32562
File: 31545 /32562
File: 31546 /32562
File: 31547 /32562
File: 31548 /32562
File: 31549 /32562
File: 31550 /32562
File: 31551 /32562
File: 31552 /32562
File: 31553 /32562
File: 31554 /32562
File: 31555 /32562
File: 31556 

File: 31944 /32562
File: 31945 /32562
File: 31946 /32562
File: 31947 /32562
File: 31948 /32562
File: 31949 /32562
File: 31950 /32562
File: 31951 /32562
File: 31952 /32562
File: 31953 /32562
File: 31954 /32562
File: 31955 /32562
File: 31956 /32562
File: 31957 /32562
File: 31958 /32562
File: 31959 /32562
File: 31960 /32562
File: 31961 /32562
File: 31962 /32562
File: 31963 /32562
File: 31964 /32562
File: 31965 /32562
File: 31966 /32562
File: 31967 /32562
File: 31968 /32562
File: 31969 /32562
File: 31970 /32562
File: 31971 /32562
File: 31972 /32562
File: 31973 /32562
File: 31974 /32562
File: 31975 /32562
File: 31976 /32562
File: 31977 /32562
File: 31978 /32562
File: 31979 /32562
File: 31980 /32562
File: 31981 /32562
File: 31982 /32562
File: 31983 /32562
File: 31984 /32562
File: 31985 /32562
File: 31986 /32562
File: 31987 /32562
File: 31988 /32562
File: 31989 /32562
File: 31990 /32562
File: 31991 /32562
File: 31992 /32562
File: 31993 /32562
File: 31994 /32562
File: 31995 /32562
File: 31996 

File: 32382 /32562
File: 32383 /32562
File: 32384 /32562
File: 32385 /32562
File: 32386 /32562
File: 32387 /32562
File: 32388 /32562
File: 32389 /32562
File: 32390 /32562
File: 32391 /32562
File: 32392 /32562
File: 32393 /32562
File: 32394 /32562
File: 32395 /32562
File: 32396 /32562
File: 32397 /32562
File: 32398 /32562
File: 32399 /32562
File: 32400 /32562
File: 32401 /32562
File: 32402 /32562
File: 32403 /32562
File: 32404 /32562
File: 32405 /32562
File: 32406 /32562
File: 32407 /32562
File: 32408 /32562
File: 32409 /32562
File: 32410 /32562
File: 32411 /32562
File: 32412 /32562
File: 32413 /32562
File: 32414 /32562
File: 32415 /32562
File: 32416 /32562
File: 32417 /32562
File: 32418 /32562
File: 32419 /32562
File: 32420 /32562
File: 32421 /32562
File: 32422 /32562
File: 32423 /32562
File: 32424 /32562
File: 32425 /32562
File: 32426 /32562
File: 32427 /32562
File: 32428 /32562
File: 32429 /32562
File: 32430 /32562
File: 32431 /32562
File: 32432 /32562
File: 32433 /32562
File: 32434 

In [17]:
# Mostramos df1
df1

,date,MPT0,MPT1,MPT2,tendencia
0,2021-05-06 10:46:29,57067.10,57068.71,57121.31,-1
1,2021-05-06 10:47:29,57019.99,57067.10,57068.71,-1
2,2021-05-06 10:48:29,57019.99,57019.99,57067.10,-1
3,2021-05-06 10:49:29,57060.60,57019.99,57019.99,-1
4,2021-05-06 10:50:29,56973.62,57060.60,57019.99,-1
...,...,...,...,...,...
32551,2021-06-09 15:37:51,34864.88,34921.01,34939.99,1
32552,2021-06-09 15:38:51,34847.33,34864.88,34921.01,1
32553,2021-06-09 15:39:51,34852.33,34847.33,34864.88,1
32554,2021-06-09 15:40:51,34878.63,34852.33,34847.33,1


In [18]:
# Guardamos datos 1
df1 = df1.reset_index(drop = True)
df1.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\datos1.csv', index = False, header=True)

### Midprice

In [20]:
# Guardamos midprice
midprice = df1[['date', 'MPT0']]
midprice.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\midprice.csv', index = False, header=True)

# Planteamiento 2

In [26]:
# Creamos los datos del planteamiento 2
cubes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 
         1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 
         2.6, 2.8, 3.0, 3.2, 3.6, 4.0, 4.4, 4.8, 5.4, 6.0] 

CrudeDf2 = createMatrix(tipo = 2, BTCOB = obBtcFiles, cubes = cubes)

File: 0 /32562
File: 1 /32562
File: 2 /32562
File: 3 /32562
File: 4 /32562
File: 5 /32562
File: 6 /32562
File: 7 /32562
File: 8 /32562
File: 9 /32562
File: 10 /32562
File: 11 /32562
File: 12 /32562
File: 13 /32562
File: 14 /32562
File: 15 /32562
File: 16 /32562
File: 17 /32562
File: 18 /32562
File: 19 /32562
File: 20 /32562
File: 21 /32562
File: 22 /32562
File: 23 /32562
File: 24 /32562
File: 25 /32562
File: 26 /32562
File: 27 /32562
File: 28 /32562
File: 29 /32562
File: 30 /32562
File: 31 /32562
File: 32 /32562
File: 33 /32562
File: 34 /32562
File: 35 /32562
File: 36 /32562
File: 37 /32562
File: 38 /32562
File: 39 /32562
File: 40 /32562
File: 41 /32562
File: 42 /32562
File: 43 /32562
File: 44 /32562
File: 45 /32562
File: 46 /32562
File: 47 /32562
File: 48 /32562
File: 49 /32562
File: 50 /32562
File: 51 /32562
File: 52 /32562
File: 53 /32562
File: 54 /32562
File: 55 /32562
File: 56 /32562
File: 57 /32562
File: 58 /32562
File: 59 /32562
File: 60 /32562
File: 61 /32562
File: 62 /32562
Fi

File: 489 /32562
File: 490 /32562
File: 491 /32562
File: 492 /32562
File: 493 /32562
File: 494 /32562
File: 495 /32562
File: 496 /32562
File: 497 /32562
File: 498 /32562
File: 499 /32562
File: 500 /32562
File: 501 /32562
File: 502 /32562
File: 503 /32562
File: 504 /32562
File: 505 /32562
File: 506 /32562
File: 507 /32562
File: 508 /32562
File: 509 /32562
File: 510 /32562
File: 511 /32562
File: 512 /32562
File: 513 /32562
File: 514 /32562
File: 515 /32562
File: 516 /32562
File: 517 /32562
File: 518 /32562
File: 519 /32562
File: 520 /32562
File: 521 /32562
File: 522 /32562
File: 523 /32562
File: 524 /32562
File: 525 /32562
File: 526 /32562
File: 527 /32562
File: 528 /32562
File: 529 /32562
File: 530 /32562
File: 531 /32562
File: 532 /32562
File: 533 /32562
File: 534 /32562
File: 535 /32562
File: 536 /32562
File: 537 /32562
File: 538 /32562
File: 539 /32562
File: 540 /32562
File: 541 /32562
File: 542 /32562
File: 543 /32562
File: 544 /32562
File: 545 /32562
File: 546 /32562
File: 547 /325

File: 973 /32562
File: 974 /32562
File: 975 /32562
File: 976 /32562
File: 977 /32562
File: 978 /32562
File: 979 /32562
File: 980 /32562
File: 981 /32562
File: 982 /32562
File: 983 /32562
File: 984 /32562
File: 985 /32562
File: 986 /32562
File: 987 /32562
File: 988 /32562
File: 989 /32562
File: 990 /32562
File: 991 /32562
File: 992 /32562
File: 993 /32562
File: 994 /32562
File: 995 /32562
File: 996 /32562
File: 997 /32562
File: 998 /32562
File: 999 /32562
File: 1000 /32562
File: 1001 /32562
File: 1002 /32562
File: 1003 /32562
File: 1004 /32562
File: 1005 /32562
File: 1006 /32562
File: 1007 /32562
File: 1008 /32562
File: 1009 /32562
File: 1010 /32562
File: 1011 /32562
File: 1012 /32562
File: 1013 /32562
File: 1014 /32562
File: 1015 /32562
File: 1016 /32562
File: 1017 /32562
File: 1018 /32562
File: 1019 /32562
File: 1020 /32562
File: 1021 /32562
File: 1022 /32562
File: 1023 /32562
File: 1024 /32562
File: 1025 /32562
File: 1026 /32562
File: 1027 /32562
File: 1028 /32562
File: 1029 /32562
F

File: 1431 /32562
File: 1432 /32562
File: 1433 /32562
File: 1434 /32562
File: 1435 /32562
File: 1436 /32562
File: 1437 /32562
File: 1438 /32562
File: 1439 /32562
File: 1440 /32562
File: 1441 /32562
File: 1442 /32562
File: 1443 /32562
File: 1444 /32562
File: 1445 /32562
File: 1446 /32562
File: 1447 /32562
File: 1448 /32562
File: 1449 /32562
File: 1450 /32562
File: 1451 /32562
File: 1452 /32562
File: 1453 /32562
File: 1454 /32562
File: 1455 /32562
File: 1456 /32562
File: 1457 /32562
File: 1458 /32562
File: 1459 /32562
File: 1460 /32562
File: 1461 /32562
File: 1462 /32562
File: 1463 /32562
File: 1464 /32562
File: 1465 /32562
File: 1466 /32562
File: 1467 /32562
File: 1468 /32562
File: 1469 /32562
File: 1470 /32562
File: 1471 /32562
File: 1472 /32562
File: 1473 /32562
File: 1474 /32562
File: 1475 /32562
File: 1476 /32562
File: 1477 /32562
File: 1478 /32562
File: 1479 /32562
File: 1480 /32562
File: 1481 /32562
File: 1482 /32562
File: 1483 /32562
File: 1484 /32562
File: 1485 /32562
File: 1486

File: 1887 /32562
File: 1888 /32562
File: 1889 /32562
File: 1890 /32562
File: 1891 /32562
File: 1892 /32562
File: 1893 /32562
File: 1894 /32562
File: 1895 /32562
File: 1896 /32562
File: 1897 /32562
File: 1898 /32562
File: 1899 /32562
File: 1900 /32562
File: 1901 /32562
File: 1902 /32562
File: 1903 /32562
File: 1904 /32562
File: 1905 /32562
File: 1906 /32562
File: 1907 /32562
File: 1908 /32562
File: 1909 /32562
File: 1910 /32562
File: 1911 /32562
File: 1912 /32562
File: 1913 /32562
File: 1914 /32562
File: 1915 /32562
File: 1916 /32562
File: 1917 /32562
File: 1918 /32562
File: 1919 /32562
File: 1920 /32562
File: 1921 /32562
File: 1922 /32562
File: 1923 /32562
File: 1924 /32562
File: 1925 /32562
File: 1926 /32562
File: 1927 /32562
File: 1928 /32562
File: 1929 /32562
File: 1930 /32562
File: 1931 /32562
File: 1932 /32562
File: 1933 /32562
File: 1934 /32562
File: 1935 /32562
File: 1936 /32562
File: 1937 /32562
File: 1938 /32562
File: 1939 /32562
File: 1940 /32562
File: 1941 /32562
File: 1942

...

File: 30444 /32562
File: 30445 /32562
File: 30446 /32562
File: 30447 /32562
File: 30448 /32562
File: 30449 /32562
File: 30450 /32562
File: 30451 /32562
File: 30452 /32562
File: 30453 /32562
File: 30454 /32562
File: 30455 /32562
File: 30456 /32562
File: 30457 /32562
File: 30458 /32562
File: 30459 /32562
File: 30460 /32562
File: 30461 /32562
File: 30462 /32562
File: 30463 /32562
File: 30464 /32562
File: 30465 /32562
File: 30466 /32562
File: 30467 /32562
File: 30468 /32562
File: 30469 /32562
File: 30470 /32562
File: 30471 /32562
File: 30472 /32562
File: 30473 /32562
File: 30474 /32562
File: 30475 /32562
File: 30476 /32562
File: 30477 /32562
File: 30478 /32562
File: 30479 /32562
File: 30480 /32562
File: 30481 /32562
File: 30482 /32562
File: 30483 /32562
File: 30484 /32562
File: 30485 /32562
File: 30486 /32562
File: 30487 /32562
File: 30488 /32562
File: 30489 /32562
File: 30490 /32562
File: 30491 /32562
File: 30492 /32562
File: 30493 /32562
File: 30494 /32562
File: 30495 /32562
File: 30496 

File: 30876 /32562
File: 30877 /32562
File: 30878 /32562
File: 30879 /32562
File: 30880 /32562
File: 30881 /32562
File: 30882 /32562
File: 30883 /32562
File: 30884 /32562
File: 30885 /32562
File: 30886 /32562
File: 30887 /32562
File: 30888 /32562
File: 30889 /32562
File: 30890 /32562
File: 30891 /32562
File: 30892 /32562
File: 30893 /32562
File: 30894 /32562
File: 30895 /32562
File: 30896 /32562
File: 30897 /32562
File: 30898 /32562
File: 30899 /32562
File: 30900 /32562
File: 30901 /32562
File: 30902 /32562
File: 30903 /32562
File: 30904 /32562
File: 30905 /32562
File: 30906 /32562
File: 30907 /32562
File: 30908 /32562
File: 30909 /32562
File: 30910 /32562
File: 30911 /32562
File: 30912 /32562
File: 30913 /32562
File: 30914 /32562
File: 30915 /32562
File: 30916 /32562
File: 30917 /32562
File: 30918 /32562
File: 30919 /32562
File: 30920 /32562
File: 30921 /32562
File: 30922 /32562
File: 30923 /32562
File: 30924 /32562
File: 30925 /32562
File: 30926 /32562
File: 30927 /32562
File: 30928 

File: 31309 /32562
File: 31310 /32562
File: 31311 /32562
File: 31312 /32562
File: 31313 /32562
File: 31314 /32562
File: 31315 /32562
File: 31316 /32562
File: 31317 /32562
File: 31318 /32562
File: 31319 /32562
File: 31320 /32562
File: 31321 /32562
File: 31322 /32562
File: 31323 /32562
File: 31324 /32562
File: 31325 /32562
File: 31326 /32562
File: 31327 /32562
File: 31328 /32562
File: 31329 /32562
File: 31330 /32562
File: 31331 /32562
File: 31332 /32562
File: 31333 /32562
File: 31334 /32562
File: 31335 /32562
File: 31336 /32562
File: 31337 /32562
File: 31338 /32562
File: 31339 /32562
File: 31340 /32562
File: 31341 /32562
File: 31342 /32562
File: 31343 /32562
File: 31344 /32562
File: 31345 /32562
File: 31346 /32562
File: 31347 /32562
File: 31348 /32562
File: 31349 /32562
File: 31350 /32562
File: 31351 /32562
File: 31352 /32562
File: 31353 /32562
File: 31354 /32562
File: 31355 /32562
File: 31356 /32562
File: 31357 /32562
File: 31358 /32562
File: 31359 /32562
File: 31360 /32562
File: 31361 

File: 31742 /32562
File: 31743 /32562
File: 31744 /32562
File: 31745 /32562
File: 31746 /32562
File: 31747 /32562
File: 31748 /32562
File: 31749 /32562
File: 31750 /32562
File: 31751 /32562
File: 31752 /32562
File: 31753 /32562
File: 31754 /32562
File: 31755 /32562
File: 31756 /32562
File: 31757 /32562
File: 31758 /32562
File: 31759 /32562
File: 31760 /32562
File: 31761 /32562
File: 31762 /32562
File: 31763 /32562
File: 31764 /32562
File: 31765 /32562
File: 31766 /32562
File: 31767 /32562
File: 31768 /32562
File: 31769 /32562
File: 31770 /32562
File: 31771 /32562
File: 31772 /32562
File: 31773 /32562
File: 31774 /32562
File: 31775 /32562
File: 31776 /32562
File: 31777 /32562
File: 31778 /32562
File: 31779 /32562
File: 31780 /32562
File: 31781 /32562
File: 31782 /32562
File: 31783 /32562
File: 31784 /32562
File: 31785 /32562
File: 31786 /32562
File: 31787 /32562
File: 31788 /32562
File: 31789 /32562
File: 31790 /32562
File: 31791 /32562
File: 31792 /32562
File: 31793 /32562
File: 31794 

File: 32175 /32562
File: 32176 /32562
File: 32177 /32562
File: 32178 /32562
File: 32179 /32562
File: 32180 /32562
File: 32181 /32562
File: 32182 /32562
File: 32183 /32562
File: 32184 /32562
File: 32185 /32562
File: 32186 /32562
File: 32187 /32562
File: 32188 /32562
File: 32189 /32562
File: 32190 /32562
File: 32191 /32562
File: 32192 /32562
File: 32193 /32562
File: 32194 /32562
File: 32195 /32562
File: 32196 /32562
File: 32197 /32562
File: 32198 /32562
File: 32199 /32562
File: 32200 /32562
File: 32201 /32562
File: 32202 /32562
File: 32203 /32562
File: 32204 /32562
File: 32205 /32562
File: 32206 /32562
File: 32207 /32562
File: 32208 /32562
File: 32209 /32562
File: 32210 /32562
File: 32211 /32562
File: 32212 /32562
File: 32213 /32562
File: 32214 /32562
File: 32215 /32562
File: 32216 /32562
File: 32217 /32562
File: 32218 /32562
File: 32219 /32562
File: 32220 /32562
File: 32221 /32562
File: 32222 /32562
File: 32223 /32562
File: 32224 /32562
File: 32225 /32562
File: 32226 /32562
File: 32227 

In [27]:
CrudeDf2

,date,minBjg,bid0.1,bid0.2,bid0.3,bid0.4,bid0.5,bid0.6,bid0.7,bid0.8,...,AskSlp3.0,AskSlp3.6,AskSlp4.4,AskSlp5.4,AskMean,AskMedian,AskQ1,AskQ3,AskSkew,tendencia
0,2021-05-06 10:44:29,1004,628563.307,1248837.194,1977329.955,2164982.284,3249601.520,3452320.895,3725048.421,3797453.878,...,8112.378774,9293.058449,8282.956581,6749.075733,9.869019e+06,9.683745e+06,5530828.725,1.436910e+07,26.453756,-1
1,2021-05-06 10:45:30,1005,502854.877,1240872.462,1802990.044,2857197.556,3047893.721,3353401.664,3455659.289,3652681.867,...,7822.224113,9365.418663,8360.456056,6812.223453,1.004180e+07,9.714624e+06,5650199.940,1.437679e+07,26.143798,-1
2,2021-05-06 10:46:29,1006,611389.652,1281571.126,1891709.977,2910178.200,3071803.679,3347062.199,3448818.684,3653728.119,...,7746.365542,9301.947254,8300.637602,6763.482491,9.911164e+06,9.578896e+06,5514443.044,1.404388e+07,26.175759,-1
3,2021-05-06 10:47:29,1007,708895.817,1205316.331,1949371.726,3012301.134,3350333.425,3501523.096,3696728.764,3781040.964,...,7553.496467,9014.413441,8182.907307,6669.683404,9.826127e+06,9.494710e+06,5268943.691,1.387055e+07,26.345092,-1
4,2021-05-06 10:48:29,1008,818472.599,1428503.890,2085845.295,3062058.738,3410785.625,3526624.563,3728829.223,3805673.667,...,7630.111431,9078.259243,8234.365647,6709.483120,9.945798e+06,9.611191e+06,5377786.522,1.398936e+07,26.310712,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32553,2021-06-09 15:37:51,1297,244233.965,819177.039,1041969.127,1954555.446,2064507.201,2089520.140,2335510.009,3561145.850,...,8303.753055,8268.467016,7378.538128,6739.439676,1.017580e+07,9.898603e+06,5269726.820,1.262798e+07,23.275919,1
32554,2021-06-09 15:38:51,1298,422521.808,1011574.741,2038185.120,2057859.097,2148390.655,2277551.870,3533923.788,3714107.275,...,8049.070052,8146.605479,7305.767985,6679.761271,9.930274e+06,9.803363e+06,5166815.722,1.252174e+07,23.335638,1
32555,2021-06-09 15:39:51,1299,311800.968,873616.535,1147931.944,1164714.837,1270361.289,1398565.203,2365849.510,3604227.335,...,8286.903397,8459.989477,7565.106763,6885.052658,1.030758e+07,1.018776e+07,5095100.917,1.286537e+07,23.055003,1
32556,2021-06-09 15:40:51,1300,440563.780,795742.882,1187698.824,1367479.750,1384391.712,1478338.174,2546565.389,3794239.881,...,8307.200400,8269.478293,7423.883391,6744.624444,1.001322e+07,9.892765e+06,4805492.090,1.257890e+07,23.244730,1


In [28]:
# Guardamos los datos del planteamiento 2
CrudeDf2 = CrudeDf2.reset_index(drop = True)
CrudeDf2.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\CrudeDatos2.csv', index = False, header=True)

### Planteamiento 2: PCA

Del script de reducción de dimensionalidad

In [29]:
# Cargamos los datos del planteamiento 2
dfCrude2 = pd.read_csv('datos\CrudeDatos2.csv')

# Eliminamos las variables sustituidas por las componentes principales
idx = list(range(2, 32)) + list(range(34, 49)) + list(range(54, 84)) + list(range(86, 101)) 
X2 = dfCrude2.drop(dfCrude2.columns[idx], axis = 1)

In [30]:
# Cargamos PCA de bitcoin
BTC_PCA = pd.read_csv('datos\BTC_PCA_Vars.csv')

In [31]:
# Concatenamos las variables no reducidas con las variables de pca
df2PCA = pd.concat([X2, BTC_PCA], axis=1)

In [32]:
# GUardamos los resultados
df2PCA = df2PCA.reset_index(drop = True)
df2PCA.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\datos2.csv', index = False, header=True)

## Planteamiento 3

### Memoria de los datos

In [33]:
def memory(X):
    
    originalNames = X.columns.values
    X.columns = originalNames + '_T0'
    
    # t-{1}
    NamesBlock2 = originalNames + '_T1'
    for t in NamesBlock2:
        X[t] = np.nan

    # t-{2}
    NamesBlock3 = originalNames + '_T2'
    for t in NamesBlock3:
        X[t] = np.nan
        
    for i in range(2, len(X)):
        # t-{1}
        X.iloc[i, 16:32] = X.iloc[i - 1, 0:16].values
        
        # t-{2}
        X.iloc[i, 32:48] = X.iloc[i - 2, 0:16].values
        
    # Eliminamos los 2 primeros registros porque no tienen memoria
    
    return(X.iloc[2:, :])

In [34]:
# Cargamos los datos de PCA bitcoin
BTC_PCA = pd.read_csv('datos/BTC_PCA_Vars.csv')

In [35]:
# Aplicamos memoria a los datos 
BTC_PCA_Memory = memory(BTC_PCA)

### Planteamiento 3: PCA

In [37]:
# Cargamos el modelo 2
df2 = pd.read_csv('datos\CrudeDatos2.csv')

# Eliminamos las variables incluidas en PCA
idx = list(range(2, 32)) + list(range(34, 49)) + list(range(54, 84)) + list(range(86, 101)) 
X2 = df2.drop(df2.columns[idx], axis = 1)

# Eliminamos los dos primeros registros por no memoria
X2 = X2.iloc[2:, :]

In [39]:
# Concatenamos BTC_PCA_Memory y X2
df3 = pd.concat([BTC_PCA_Memory, X2], axis=1).reset_index(drop=True)

In [40]:
df3

,BidPCA1_T0,BidPCA2_T0,BidPCA3_T0,BidPCA4_T0,AskPCA1_T0,AskPCA2_T0,AskPCA3_T0,AskPCA4_T0,BidSlopePCA1_T0,BidSlopePCA2_T0,...,BidQ3,BidSkew,WMAsk,WStAsk,AskMean,AskMedian,AskQ1,AskQ3,AskSkew,tendencia
0,-5.404031e+06,-3.893743e+06,-2.439755e+05,-1.526498e+06,1.985440e+07,3.245390e+05,-5.072499e+05,1.805044e+06,-3421.262628,5640.775774,...,4171303.617,29.710379,4139.0,39584.0,9.911164e+06,9.578896e+06,5514443.044,1.404388e+07,26.175759,-1
1,-3.685499e+06,-4.591512e+06,2.442498e+05,-1.361295e+06,1.826665e+07,-6.582628e+05,-9.805237e+05,1.284599e+06,-1121.035884,6569.643156,...,4513252.809,28.082148,4060.0,39234.0,9.826127e+06,9.494710e+06,5268943.691,1.387055e+07,26.345092,-1
2,-3.665536e+06,-4.630519e+06,2.922839e+05,-1.405615e+06,1.881777e+07,-3.737041e+05,-8.432082e+05,1.382847e+06,-183.366910,8272.114621,...,4514602.690,28.102686,4086.0,39125.0,9.945798e+06,9.611191e+06,5377786.522,1.398936e+07,26.310712,-1
3,-5.020883e+06,-3.729838e+06,-1.810942e+05,-1.290307e+06,2.136660e+07,8.540589e+05,-4.332281e+05,1.658699e+06,-4612.093542,3281.115336,...,4306287.303,28.535053,4206.0,39663.0,1.027151e+07,9.936988e+06,5346283.354,1.431646e+07,25.747956,-1
4,-3.151139e+06,-4.906890e+06,5.683601e+04,-1.208226e+06,1.815527e+07,-8.589824e+05,-7.718446e+05,1.292772e+06,-108.321398,7130.597805,...,4530023.961,27.285342,4120.0,38842.0,1.000766e+07,9.704877e+06,5675090.606,1.401500e+07,26.381128,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32551,4.268670e+05,-4.178686e+06,1.089996e+05,3.406378e+05,-5.381813e+06,5.159265e+05,-2.566906e+06,-2.689152e+05,8800.949387,-5339.152062,...,5823061.053,23.988489,3596.0,31287.0,1.017580e+07,9.898603e+06,5269726.820,1.262798e+07,23.275919,1
32552,1.413410e+06,-4.978302e+06,8.499572e+05,-2.762508e+04,-6.061573e+06,1.288848e+05,-2.631849e+06,-3.597269e+05,15624.089589,1339.817932,...,6037625.039,23.922995,3538.0,31172.0,9.930274e+06,9.803363e+06,5166815.722,1.252174e+07,23.335638,1
32553,6.677338e+04,-3.519702e+06,-5.832082e+05,4.883854e+05,-4.714112e+06,4.929749e+05,-2.238271e+06,-1.117620e+05,7621.664537,-3992.223802,...,5164544.290,24.002345,3627.0,31140.0,1.030758e+07,1.018776e+07,5095100.917,1.286537e+07,23.055003,1
32554,4.461684e+05,-3.472625e+06,-5.785144e+05,3.990754e+05,-5.238230e+06,7.688089e+05,-2.341851e+06,-1.884936e+05,9034.243860,-1056.681767,...,5360598.683,23.910512,3523.0,31049.0,1.001322e+07,9.892765e+06,4805492.090,1.257890e+07,23.244730,1


In [41]:
# Guardamos el modelo
df3 = df3.reset_index(drop = True)
df3.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\datos3.csv', index = False, header=True)

## Planteamiento 4

In [42]:
def Tipo4(ETHOB, cubes):
    
    # ETHOB: Ethereum Order Book Files
    # cubes: nº de intervalos y tamaño para definir las variables: bids, asks y slope 
    
    midpriceArray = [] # for Bitcoin
    dateArray = []
    OBArray = [] # for Bitcoin
    breakfiles = []
    
    for i in range(len(ETHOB)):
        try:
            print('File:', i, '/32562')
            df_ObEth = pd.read_feather('Order_Book/ETHBUSD/' + ETHOB[i])
            
            # ============ # 
            # Preprocesado #
            # ============ #
            dfBids, dfAsks, midprice = preProcesado(df_ObEth)
            midpriceArray.append(midprice)
            
            # =================== # 
            # Add date: Create Dn #
            # =================== #
            date = addDate(df_ObEth)
            dateArray.append(date)
            
            # ============================== # 
            # Order Book features: Create Bn #
            # ============================== #
            ob = addOrderBook(dfBids, dfAsks, cubes, midprice, crp = 'eth')
            OBArray.append(ob)
        
        except:
            print('breakfile')
            breakfiles.append(i)
            continue
                
    D = pd.DataFrame(dateArray)
    D.columns = ['date', 'minBjg'] 
    
    B = pd.DataFrame(OBArray)
    B.columns = ['bid0.1_ETH', 'bid0.2_ETH', 'bid0.3_ETH', 'bid0.4_ETH', 'bid0.5_ETH',
                 'bid0.6_ETH', 'bid0.7_ETH', 'bid0.8_ETH', 'bid0.9_ETH', 'bid1.0_ETH',
                 'bid1.1_ETH', 'bid1.2_ETH', 'bid1.3_ETH', 'bid1.4_ETH', 'bid1.5_ETH', 
                 'bid1.6_ETH', 'bid1.8_ETH', 'bid2.0_ETH', 'bid2.2_ETH', 'bid2.4_ETH', 
                 'bid2.6_ETH', 'bid2.8_ETH', 'bid3.0_ETH', 'bid3.2_ETH', 'bid3.6_ETH', 
                 'bid4.0_ETH', 'bid4.4_ETH', 'bid4.8_ETH', 'bid5.4_ETH', 'bid6.0_ETH', 
                 'BidSlp0.1_ETH', 'BidSlp0.3_ETH', 'BidSlp0.5_ETH', 'BidSlp0.7_ETH', 
                 'BidSlp0.9_ETH', 'BidSlp1.1_ETH', 'BidSlp1.3_ETH', 'BidSlp1.5_ETH', 
                 'BidSlp1.8_ETH', 'BidSlp2.2_ETH', 'BidSlp2.6_ETH', 'BidSlp3.0_ETH', 
                 'BidSlp3.6_ETH', 'BidSlp4.4_ETH', 'BidSlp5.4_ETH',
                 'ask0.1_ETH', 'ask0.2_ETH', 'ask0.3_ETH', 'ask0.4_ETH', 'ask0.5_ETH', 
                 'ask0.6_ETH', 'ask0.7_ETH', 'ask0.8_ETH', 'ask0.9_ETH', 'ask1.0_ETH',
                 'ask1.1_ETH', 'ask1.2_ETH', 'ask1.3_ETH', 'ask1.4_ETH', 'ask1.5_ETH', 
                 'ask1.6_ETH', 'ask1.8_ETH', 'ask2.0_ETH', 'ask2.2_ETH', 'ask2.4_ETH', 
                 'ask2.6_ETH', 'ask2.8_ETH', 'ask3.0_ETH', 'ask3.2_ETH', 'ask3.6_ETH', 
                 'ask4.0_ETH', 'ask4.4_ETH', 'ask4.8_ETH', 'ask5.4_ETH', 'ask6.0_ETH',
                 'AskSlp0.1_ETH', 'AskSlp0.3_ETH', 'AskSlp0.5_ETH', 'AskSlp0.7_ETH', 
                 'AskSlp0.9_ETH', 'AskSlp1.1_ETH', 'AskSlp1.3_ETH', 'AskSlp1.5_ETH',
                 'AskSlp1.8_ETH', 'AskSlp2.2_ETH', 'AskSlp2.6_ETH', 'AskSlp3.0_ETH', 
                 'AskSlp3.6_ETH', 'AskSlp4.4_ETH', 'AskSlp5.4_ETH']
     
    X = pd.concat([D.reset_index(drop = True), B], axis=1)
        
    return(X)

In [43]:
cubes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 
         1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 
         2.6, 2.8, 3.0, 3.2, 3.6, 4.0, 4.4, 4.8, 5.4, 6.0] 

EthDf4 = Tipo4(ETHOB = obEthFiles, cubes = cubes)

File: 0 /32562
File: 1 /32562
File: 2 /32562
File: 3 /32562
File: 4 /32562
File: 5 /32562
File: 6 /32562
File: 7 /32562
File: 8 /32562
File: 9 /32562
File: 10 /32562
File: 11 /32562
File: 12 /32562
File: 13 /32562
File: 14 /32562
File: 15 /32562
File: 16 /32562
File: 17 /32562
File: 18 /32562
File: 19 /32562
File: 20 /32562
File: 21 /32562
File: 22 /32562
File: 23 /32562
File: 24 /32562
File: 25 /32562
File: 26 /32562
File: 27 /32562
File: 28 /32562
File: 29 /32562
File: 30 /32562
File: 31 /32562
File: 32 /32562
File: 33 /32562
File: 34 /32562
File: 35 /32562
File: 36 /32562
File: 37 /32562
File: 38 /32562
File: 39 /32562
File: 40 /32562
File: 41 /32562
File: 42 /32562
File: 43 /32562
File: 44 /32562
File: 45 /32562
File: 46 /32562
File: 47 /32562
File: 48 /32562
File: 49 /32562
File: 50 /32562
File: 51 /32562
File: 52 /32562
File: 53 /32562
File: 54 /32562
File: 55 /32562
File: 56 /32562
File: 57 /32562
File: 58 /32562
File: 59 /32562
File: 60 /32562
File: 61 /32562
File: 62 /32562
Fi

File: 492 /32562
File: 493 /32562
File: 494 /32562
File: 495 /32562
File: 496 /32562
File: 497 /32562
File: 498 /32562
File: 499 /32562
File: 500 /32562
File: 501 /32562
File: 502 /32562
File: 503 /32562
File: 504 /32562
File: 505 /32562
File: 506 /32562
File: 507 /32562
File: 508 /32562
File: 509 /32562
File: 510 /32562
File: 511 /32562
File: 512 /32562
File: 513 /32562
File: 514 /32562
File: 515 /32562
File: 516 /32562
File: 517 /32562
File: 518 /32562
File: 519 /32562
File: 520 /32562
File: 521 /32562
File: 522 /32562
File: 523 /32562
File: 524 /32562
File: 525 /32562
File: 526 /32562
File: 527 /32562
File: 528 /32562
File: 529 /32562
File: 530 /32562
File: 531 /32562
File: 532 /32562
File: 533 /32562
File: 534 /32562
File: 535 /32562
File: 536 /32562
File: 537 /32562
File: 538 /32562
File: 539 /32562
File: 540 /32562
File: 541 /32562
File: 542 /32562
File: 543 /32562
File: 544 /32562
File: 545 /32562
File: 546 /32562
File: 547 /32562
File: 548 /32562
File: 549 /32562
File: 550 /325

File: 976 /32562
File: 977 /32562
File: 978 /32562
File: 979 /32562
File: 980 /32562
File: 981 /32562
File: 982 /32562
File: 983 /32562
File: 984 /32562
File: 985 /32562
File: 986 /32562
File: 987 /32562
File: 988 /32562
File: 989 /32562
File: 990 /32562
File: 991 /32562
File: 992 /32562
File: 993 /32562
File: 994 /32562
File: 995 /32562
File: 996 /32562
File: 997 /32562
File: 998 /32562
File: 999 /32562
File: 1000 /32562
File: 1001 /32562
File: 1002 /32562
File: 1003 /32562
File: 1004 /32562
File: 1005 /32562
File: 1006 /32562
File: 1007 /32562
File: 1008 /32562
File: 1009 /32562
File: 1010 /32562
File: 1011 /32562
File: 1012 /32562
File: 1013 /32562
File: 1014 /32562
File: 1015 /32562
File: 1016 /32562
File: 1017 /32562
File: 1018 /32562
File: 1019 /32562
File: 1020 /32562
File: 1021 /32562
File: 1022 /32562
File: 1023 /32562
File: 1024 /32562
File: 1025 /32562
File: 1026 /32562
File: 1027 /32562
File: 1028 /32562
File: 1029 /32562
File: 1030 /32562
File: 1031 /32562
File: 1032 /3256

File: 1435 /32562
File: 1436 /32562
File: 1437 /32562
File: 1438 /32562
File: 1439 /32562
File: 1440 /32562
File: 1441 /32562
File: 1442 /32562
File: 1443 /32562
File: 1444 /32562
File: 1445 /32562
File: 1446 /32562
File: 1447 /32562
File: 1448 /32562
File: 1449 /32562
File: 1450 /32562
File: 1451 /32562
File: 1452 /32562
File: 1453 /32562
File: 1454 /32562
File: 1455 /32562
File: 1456 /32562
File: 1457 /32562
File: 1458 /32562
File: 1459 /32562
File: 1460 /32562
File: 1461 /32562
File: 1462 /32562
File: 1463 /32562
File: 1464 /32562
File: 1465 /32562
File: 1466 /32562
File: 1467 /32562
File: 1468 /32562
File: 1469 /32562
File: 1470 /32562
File: 1471 /32562
File: 1472 /32562
File: 1473 /32562
File: 1474 /32562
File: 1475 /32562
File: 1476 /32562
File: 1477 /32562
File: 1478 /32562
File: 1479 /32562
File: 1480 /32562
File: 1481 /32562
File: 1482 /32562
File: 1483 /32562
File: 1484 /32562
File: 1485 /32562
File: 1486 /32562
File: 1487 /32562
File: 1488 /32562
File: 1489 /32562
File: 1490

File: 1892 /32562
File: 1893 /32562
File: 1894 /32562
File: 1895 /32562
File: 1896 /32562
File: 1897 /32562
File: 1898 /32562
File: 1899 /32562
File: 1900 /32562
File: 1901 /32562
File: 1902 /32562
File: 1903 /32562
File: 1904 /32562
File: 1905 /32562
File: 1906 /32562
File: 1907 /32562
File: 1908 /32562
File: 1909 /32562
File: 1910 /32562
File: 1911 /32562
File: 1912 /32562
File: 1913 /32562
File: 1914 /32562
File: 1915 /32562
File: 1916 /32562
File: 1917 /32562
File: 1918 /32562
File: 1919 /32562
File: 1920 /32562
File: 1921 /32562
File: 1922 /32562
File: 1923 /32562
File: 1924 /32562
File: 1925 /32562
File: 1926 /32562
File: 1927 /32562
File: 1928 /32562
File: 1929 /32562
File: 1930 /32562
File: 1931 /32562
File: 1932 /32562
File: 1933 /32562
File: 1934 /32562
File: 1935 /32562
File: 1936 /32562
File: 1937 /32562
File: 1938 /32562
File: 1939 /32562
File: 1940 /32562
File: 1941 /32562
File: 1942 /32562
File: 1943 /32562
File: 1944 /32562
File: 1945 /32562
File: 1946 /32562
File: 1947

...

File: 30445 /32562
File: 30446 /32562
File: 30447 /32562
File: 30448 /32562
File: 30449 /32562
File: 30450 /32562
File: 30451 /32562
File: 30452 /32562
File: 30453 /32562
File: 30454 /32562
File: 30455 /32562
File: 30456 /32562
File: 30457 /32562
File: 30458 /32562
File: 30459 /32562
File: 30460 /32562
File: 30461 /32562
File: 30462 /32562
File: 30463 /32562
File: 30464 /32562
File: 30465 /32562
File: 30466 /32562
File: 30467 /32562
File: 30468 /32562
File: 30469 /32562
File: 30470 /32562
File: 30471 /32562
File: 30472 /32562
File: 30473 /32562
File: 30474 /32562
File: 30475 /32562
File: 30476 /32562
File: 30477 /32562
File: 30478 /32562
File: 30479 /32562
File: 30480 /32562
File: 30481 /32562
File: 30482 /32562
File: 30483 /32562
File: 30484 /32562
File: 30485 /32562
File: 30486 /32562
File: 30487 /32562
File: 30488 /32562
File: 30489 /32562
File: 30490 /32562
File: 30491 /32562
File: 30492 /32562
File: 30493 /32562
File: 30494 /32562
File: 30495 /32562
File: 30496 /32562
File: 30497 

File: 30879 /32562
File: 30880 /32562
File: 30881 /32562
File: 30882 /32562
File: 30883 /32562
File: 30884 /32562
File: 30885 /32562
File: 30886 /32562
File: 30887 /32562
File: 30888 /32562
File: 30889 /32562
File: 30890 /32562
File: 30891 /32562
File: 30892 /32562
File: 30893 /32562
File: 30894 /32562
File: 30895 /32562
File: 30896 /32562
File: 30897 /32562
File: 30898 /32562
File: 30899 /32562
File: 30900 /32562
File: 30901 /32562
File: 30902 /32562
File: 30903 /32562
File: 30904 /32562
File: 30905 /32562
File: 30906 /32562
File: 30907 /32562
File: 30908 /32562
File: 30909 /32562
File: 30910 /32562
File: 30911 /32562
File: 30912 /32562
File: 30913 /32562
File: 30914 /32562
File: 30915 /32562
File: 30916 /32562
File: 30917 /32562
File: 30918 /32562
File: 30919 /32562
File: 30920 /32562
File: 30921 /32562
File: 30922 /32562
File: 30923 /32562
File: 30924 /32562
File: 30925 /32562
File: 30926 /32562
File: 30927 /32562
File: 30928 /32562
File: 30929 /32562
File: 30930 /32562
File: 30931 

File: 31313 /32562
File: 31314 /32562
File: 31315 /32562
File: 31316 /32562
File: 31317 /32562
File: 31318 /32562
File: 31319 /32562
File: 31320 /32562
File: 31321 /32562
File: 31322 /32562
File: 31323 /32562
File: 31324 /32562
File: 31325 /32562
File: 31326 /32562
File: 31327 /32562
File: 31328 /32562
File: 31329 /32562
File: 31330 /32562
File: 31331 /32562
File: 31332 /32562
File: 31333 /32562
File: 31334 /32562
File: 31335 /32562
File: 31336 /32562
File: 31337 /32562
File: 31338 /32562
File: 31339 /32562
File: 31340 /32562
File: 31341 /32562
File: 31342 /32562
File: 31343 /32562
File: 31344 /32562
File: 31345 /32562
File: 31346 /32562
File: 31347 /32562
File: 31348 /32562
File: 31349 /32562
File: 31350 /32562
File: 31351 /32562
File: 31352 /32562
File: 31353 /32562
File: 31354 /32562
File: 31355 /32562
File: 31356 /32562
File: 31357 /32562
File: 31358 /32562
File: 31359 /32562
File: 31360 /32562
File: 31361 /32562
File: 31362 /32562
File: 31363 /32562
File: 31364 /32562
File: 31365 

File: 31748 /32562
File: 31749 /32562
File: 31750 /32562
File: 31751 /32562
File: 31752 /32562
File: 31753 /32562
File: 31754 /32562
File: 31755 /32562
File: 31756 /32562
File: 31757 /32562
File: 31758 /32562
File: 31759 /32562
File: 31760 /32562
File: 31761 /32562
File: 31762 /32562
File: 31763 /32562
File: 31764 /32562
File: 31765 /32562
File: 31766 /32562
File: 31767 /32562
File: 31768 /32562
File: 31769 /32562
File: 31770 /32562
File: 31771 /32562
File: 31772 /32562
File: 31773 /32562
File: 31774 /32562
File: 31775 /32562
File: 31776 /32562
File: 31777 /32562
File: 31778 /32562
File: 31779 /32562
File: 31780 /32562
File: 31781 /32562
File: 31782 /32562
File: 31783 /32562
File: 31784 /32562
File: 31785 /32562
File: 31786 /32562
File: 31787 /32562
File: 31788 /32562
File: 31789 /32562
File: 31790 /32562
File: 31791 /32562
File: 31792 /32562
File: 31793 /32562
File: 31794 /32562
File: 31795 /32562
File: 31796 /32562
File: 31797 /32562
File: 31798 /32562
File: 31799 /32562
File: 31800 

File: 32181 /32562
File: 32182 /32562
File: 32183 /32562
File: 32184 /32562
File: 32185 /32562
File: 32186 /32562
File: 32187 /32562
File: 32188 /32562
File: 32189 /32562
File: 32190 /32562
File: 32191 /32562
File: 32192 /32562
File: 32193 /32562
File: 32194 /32562
File: 32195 /32562
File: 32196 /32562
File: 32197 /32562
File: 32198 /32562
File: 32199 /32562
File: 32200 /32562
File: 32201 /32562
File: 32202 /32562
File: 32203 /32562
File: 32204 /32562
File: 32205 /32562
File: 32206 /32562
File: 32207 /32562
File: 32208 /32562
File: 32209 /32562
File: 32210 /32562
File: 32211 /32562
File: 32212 /32562
File: 32213 /32562
File: 32214 /32562
File: 32215 /32562
File: 32216 /32562
File: 32217 /32562
File: 32218 /32562
File: 32219 /32562
File: 32220 /32562
File: 32221 /32562
File: 32222 /32562
File: 32223 /32562
File: 32224 /32562
File: 32225 /32562
File: 32226 /32562
File: 32227 /32562
File: 32228 /32562
File: 32229 /32562
File: 32230 /32562
File: 32231 /32562
File: 32232 /32562
File: 32233 

In [44]:
# Guardamos los datos de Ethereum
EthDf4 = EthDf4.reset_index(drop = True)
EthDf4.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\EthDatos4.csv', index = False, header=True)

### Planteamiento 4: PCA

In [58]:
# Cargamos los datos de Ethereum PCA
ETH_PCA = pd.read_csv('datos/ETH_PCA_Vars.csv')

In [54]:
# Aplicamos memoria
ETH_PCA_Memory = memory(ETH_PCA).reset_index(drop = True)

In [55]:
# Cargamos los datos 3
df3 = pd.read_csv('datos\datos3.csv')

In [56]:
# Concatenamos ETH_PCA_Memory y df4
df4 = pd.concat([ETH_PCA_Memory, df3], axis=1)

In [95]:
df4 = df4.reset_index(drop = True)
df4.to_csv(r'C:\Users\victo\Desktop\UNIVERSIDAD\MASTER\TFM\datos\datos4.csv', index = False, header=True)